In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import sys
sys.path.append('../src')

from support_cleaning import check_nan

Importing all the necessary libraries and functions

In [9]:
moon_phases= pd.read_csv('../data/clean/moon_phases_clean.csv')
eclipses= pd.read_csv('../data/clean/eclipses_clean.csv')
holidays= pd.read_csv('../data/clean/holidays_clean.csv')
births= pd.read_csv('../data/clean/births_clean.csv')

Bringing the necessary data 

In [3]:
moon_phases.head()

,year,month,day,phase
0,1994,1,1,waning gibbous
1,1994,1,2,waning gibbous
2,1994,1,3,waning gibbous
3,1994,1,4,last quarter
4,1994,1,5,last quarter


In [4]:
eclipses.head()

,eclipse,year,month,day
0,Solar Eclipse,1994,5,10
1,Lunar Eclipse,1994,5,25
2,Solar Eclipse,1994,11,3
3,Lunar Eclipse,1994,11,18
4,Lunar Eclipse,1995,4,15


In [7]:
holidays.head()

,holiday,year,month,day
0,"Martin Luther King, Jr. Day",1994,1,17
1,Washington's Birthday,1994,2,21
2,Good Friday,1994,4,1
3,Good Friday,1994,4,1
4,Memorial Day,1994,5,30


In [10]:
births.head()

,year,month,day,dow,births
0,1994,1,1,Saturday,8096
1,1994,1,2,Sunday,7772
2,1994,1,3,Monday,10142
3,1994,1,4,Tuesday,11248
4,1994,1,5,Wednesday,11053


Making sure the format is consistent across all dataframes. 

In [23]:
final = births.merge(moon_phases, on=['year', 'month', 'day'], how='outer') \
                    .merge(eclipses, on=['year', 'month', 'day'], how='outer') \
                    .merge(holidays, on=['year', 'month', 'day'], how='outer')

In [25]:
final.head()

,year,month,day,dow,births,phase,eclipse,holiday
0,1994,1,1,Saturday,8096,waning gibbous,NaN,NaN
1,1994,1,2,Sunday,7772,waning gibbous,NaN,NaN
2,1994,1,3,Monday,10142,waning gibbous,NaN,NaN
3,1994,1,4,Tuesday,11248,last quarter,NaN,NaN
4,1994,1,5,Wednesday,11053,last quarter,NaN,NaN


Merged all the dataframes together. Dataframe births and moon_phases had the same amount of rows. However, dataframe eclipse and holidays had less rows. Only the days where there was an eclipse in the case of eclipses df and only the days where there was a public holiday in the case of holidays. 

In [26]:
for index, row in final.iterrows():
    if pd.isna(row['eclipse']):
        final.loc[index, 'eclipse']= final.loc[index, 'phase']

Filled in the column eclipse with the value in column phase only for the nulls in column eclipse. That way I can have all the information in one column. 

In [28]:
final=final.drop('phase', axis=1)

Dropped column 'phase' since all the info is now in column 'eclipse'

In [29]:
final=final.rename(columns={'eclipse':'moon_phase'})

Changed the name of column 'eclipse' to 'moon_phase'

In [31]:
final['holiday'].fillna('Not a holiday', inplace=True)


Filled in the null values in column 'holiday' with 'Not a holiday'

In [34]:
final.head()

,year,month,day,dow,births,moon_phase,holiday
0,1994,1,1,Saturday,8096,waning gibbous,Not a holiday
1,1994,1,2,Sunday,7772,waning gibbous,Not a holiday
2,1994,1,3,Monday,10142,waning gibbous,Not a holiday
3,1994,1,4,Tuesday,11248,last quarter,Not a holiday
4,1994,1,5,Wednesday,11053,last quarter,Not a holiday


In [36]:
final.to_csv('../data/clean/final_clean.csv', index=False)

Saved file